In [2]:
import sys
#!{sys.executable} -m conda install albumentations!{sys.executable} -m conda install albumentations --yes

#!{sys.executable} -m conda init bash
!{sys.executable} -m conda activate myenv 
!{sys.executable} -m conda env list


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.


# conda environments:
#
                         /home/maftouni/anaconda3
                         /home/maftouni/anaconda3/envs/AGS
                         /home/maftouni/anaconda3/envs/myenv
                         /home/maftouni/anaconda3/envs/myenv-2
base                  *  /home/smartlab/anaconda3
3dpartseg                /home/smartlab/anaconda3/envs/3dpartseg
dh_seg                   /home/smartlab/anaconda3/envs/dh_seg
keras-seg                /home/smartlab/anaconda3/envs/keras-seg
keras_v1                 /home/smartlab/anaconda3/envs/keras_v1
pointnet2_env            /home/smartlab/anaco

In [ ]:
%matplotlib inline
from pylab import *
import cv2

In [ ]:
rcParams['figure.figsize'] = 10, 10

In [ ]:
from dataset import load_image

In [ ]:
import torch

In [ ]:
from utils import cuda

In [ ]:
from generate_masks import get_model

In [ ]:
from albumentations import Compose, Normalize

In [ ]:
from albumentations.pytorch.functional import img_to_tensor

In [ ]:
def img_transform(p=1):
    return Compose([
        Normalize(p=1)
    ], p=p)


In [ ]:
def mask_overlay(image, mask, color=(0, 255, 0)):
    """
    Helper function to visualize mask on the top of the car
    """
    mask = np.dstack((mask, mask, mask)) * np.array(color)
    mask = mask.astype(np.uint8)
    weighted_sum = cv2.addWeighted(mask, 0.5, image, 0.5, 0.)
    img = image.copy()
    ind = mask[:, :, 1] > 0    
    img[ind] = weighted_sum[ind]    
    return img

In [ ]:
#model_path = 'models/unet11_binary_20/model_0.pt'
model_path='runs/debug/model_1.pt'
model = get_model(model_path, model_type='UNet11', problem_type='binary')

In [ ]:
img_file_name = 'data/cropped_train/instrument_dataset_1/images/00162.jpg'
#img_file_name='images/frames/original.png'
gt_file_name = 'data/cropped_train/instrument_dataset_1/binary_masks/00162.png'
#gt_file_name='images/frames/mask.png'

In [ ]:
image = load_image(img_file_name)
gt = cv2.imread(gt_file_name, 0) > 0

In [ ]:
imshow(image)
print(len(image.shape))
isinstance(image,np.ndarray)

In [ ]:
with torch.no_grad():
    input_image = torch.unsqueeze(img_to_tensor(img_transform(p=1)(image=image)['image']).cuda(), dim=0)

In [ ]:
mask = model(input_image)

In [ ]:
mask_array = mask.data[0].cpu().numpy()[0]
print(mask.data[0].cpu().numpy())
#from PIL import Image
img = Image.fromarray(mask_array, 'RGB')
import matplotlib.pyplot as plt
plt.imshow(mask_array)

In [ ]:
imshow((mask_array > 0.2).astype(np.uint8))
#imshow(mask_overlay(image, (mask_array > 0).astype(np.uint8)))

In [ ]:
imshow(mask_overlay(image, (mask_array > 90).astype(np.uint8)))